In [ ]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

from create_MNIST_datasets import get_MNIST, plot_samples

In [ ]:
import syft as sy

In [ ]:
duet = sy.launch_duet(loopback=True)

In [ ]:
mnist_iid_train_dls = torch.load('client2.pth')

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.fc = nn.Linear(784, 10)
        
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = F.softmax(x)
        return x

model_client = CNN()

In [ ]:
x = duet.store[0].get()

In [ ]:
torch.nn.utils.vector_to_parameters(x,model_client.parameters())

In [ ]:
def loss_classifier(predictions,labels):
    
    m = nn.LogSoftmax(dim=1)
    loss = nn.NLLLoss(reduction="mean")
    
    return loss(m(predictions) ,labels.view(-1))

In [ ]:
train_data = mnist_iid_train_dls

epochs = 10
lr =0.1
optimizer=optim.SGD(model_client.parameters(),lr=lr) 
for e in range(epochs): 
    total_loss = 0
    for idx, (features,labels) in enumerate(train_data):
        optimizer.zero_grad()
        predictions= model_client(features)
        loss=loss_classifier(predictions,labels)
        total_loss+=loss;
        loss.backward()
        optimizer.step()
    print(total_loss)


In [ ]:
parameters = torch.nn.utils.parameters_to_vector(model_client.parameters())

In [ ]:
parameters.send(duet)

In [ ]:
duet.store.pandas

In [ ]:
duet.requests.add_handler(action="accept")